In [2]:
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDRegressor

In [3]:
train = pd.read_csv('clean_data/train_clean_v1.csv')
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,word_count,char_count,avg_word_len,stopwords,numerics,upper
0,0000997932d777bf,explanation edit make username hardcore metall...,0,0,0,0,0,0,42,264,5.162671,14,0,2
1,000103f0d9cfb60f,daww match background colour im seemingly stic...,0,0,0,0,0,0,18,112,5.587907,1,1,1
2,000113f07ec002fd,hey man im really try edit war guy constantly ...,0,0,0,0,0,0,42,233,4.571320,18,0,0
3,0001b41b1c6bb37e,cant make real suggestion improvement wonder s...,0,0,0,0,0,0,112,622,4.486686,49,0,5
4,0001d958c54c6e35,you sir hero chance remember page thats on,0,0,0,0,0,0,13,67,4.230444,4,0,0


In [4]:
test = pd.read_csv('clean_data/test_clean_v1.csv')
test.head()

,id,comment_text,word_count,char_count,avg_word_len,stopwords,numerics,upper
0,00001cee341fdb12,yo bitch ja rule succesful ever whats hat sad ...,72,367,4.111054,26,0,0
1,0000247867823ef7,rfc title fine be imo,13,50,2.999750,3,0,1
2,00013b17ad220c46,source zawe ashton lapland,16,54,2.916424,1,0,0
3,00017563c3f7919a,if look back source information update correct...,38,205,4.420936,19,0,3
4,00017695ad8997eb,anonymously edit article all,7,41,4.999286,2,0,1


In [5]:
print(train.shape)
print(test.shape)

(159571, 14)
(153164, 8)


In [6]:
train.fillna('',inplace=True)
test.fillna('',inplace=True)

In [7]:
# toxic = train[(train['toxic']==1)|(train['severe_toxic']==1)|(train['obscene']==1)|(train['threat']==1)|(train['insult']==1)|(train['identity_hate']==1)]

In [8]:
toxic_list = list()
for i, row  in train.iterrows():
    toxic_list.append(row['comment_text'].split())

In [9]:
full_toxic_list = list()
for val in toxic_list:
    for word in val:
        full_toxic_list.append(word)

In [10]:
toxic_dict = dict(Counter(full_toxic_list))

In [13]:
len(toxic_dict)

230602

#### Threshold chosen as 3 or above

In [14]:
toxic_dict = {k:v for k, v in toxic_dict.items() if v > 2}

In [16]:
len(toxic_dict)

55381

In [17]:
words = list()
for k, v  in toxic_dict.items():
    words.append(k)

In [19]:
def word_filter(x):
    temp = list()
    lines = ''
    for val in x.split():
        if val in words:
            temp.append(val)
    for val in temp:
        lines += str(val) + ' '
    return lines    

In [20]:
for i, row in train.iterrows():
    train.at[i, 'comment_text'] = word_filter(row['comment_text'])

In [21]:
for i, row in test.iterrows():
    test.at[i, 'comment_text'] = word_filter(row['comment_text'])

KeyboardInterrupt: 

In [ ]:
vectorizer = TfidfVectorizer(min_df=1,norm='l2',smooth_idf=True,use_idf=True,ngram_range=(1,1))
features = vectorizer.fit(train['comment_text'])

In [ ]:
tfidf_train = vectorizer.transform(train['comment_text'])

In [ ]:
tfidf_test = vectorizer.transform(test['comment_text'])

In [ ]:
clf1 = LogisticRegression().fit(tfidf_train, train['toxic'])
clf2 = LogisticRegression().fit(tfidf_train, train['severe_toxic'])
clf3 = LogisticRegression().fit(tfidf_train, train['obscene'])
clf4 = LogisticRegression().fit(tfidf_train, train['threat'])
clf5 = LogisticRegression().fit(tfidf_train, train['insult'])
clf6 = LogisticRegression().fit(tfidf_train, train['identity_hate'])

In [ ]:
test['toxic'] = clf1.predict_proba(tfidf_test)[:,1]
test['severe_toxic'] = clf2.predict_proba(tfidf_test)[:,1]
test['obscene'] = clf3.predict_proba(tfidf_test)[:,1]
test['threat'] = clf4.predict_proba(tfidf_test)[:,1]
test['insult'] = clf5.predict_proba(tfidf_test)[:,1]
test['identity_hate'] = clf6.predict_proba(tfidf_test)[:,1]

In [ ]:
# sub1 only 0 1 logistic regression completer failure ngram 12
# sub2 logictic regresson predict proba ngram 12
# sub3 logictic regresson predict proba ngram 13
# sub4 logictic regresson predict proba ngram 11
# sub5 BernoulliNB predict proba ngram 11
# sub6 MultinomialNB predict proba ngram 11
# sub7 SGDRegressor predict proba ngram 11
# sub8 average of 2,3,4

In [ ]:
test[['id','toxic','severe_toxic','obscene','threat','insult','identity_hate']].to_csv('sub9.csv',index=False)

In [ ]:
test[['id','toxic','severe_toxic','obscene','threat','insult','identity_hate']]

In [ ]:
print(clf1.predict_proba(tfidf_test)[:,1])